In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler, LabelEncoder
import pickle

In [2]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [4]:
df.shape

(10000, 14)

In [5]:
df.drop_duplicates(inplace=True)


In [6]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [7]:
label_encoder=LabelEncoder()
df['Gender']=label_encoder.fit_transform(df['Gender'])

In [8]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(sparse_output=False)
goe_encoder=ohe.fit_transform(df[['Geography']])

In [9]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_encoder_df=pd.DataFrame(goe_encoder,columns=['Geography_France', 'Geography_Germany', 'Geography_Spain'])

In [11]:
df=pd.concat([df,geo_encoder_df],axis=1)

In [12]:
#save the encoder and sclaer 

with open ('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder,file)
    

with open ('one_hot_encoder.pkl','wb') as file:
    pickle.dump(ohe,file)



In [13]:
x=df.drop(['Exited','Geography'],axis=1)
y=df['Exited']

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=100)

In [15]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.fit_transform(x_test)

In [16]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [17]:
x_test

array([[-0.20156328,  0.91186722, -1.51996494, ..., -1.01952389,
         1.76660101, -0.57375783],
       [-0.58842796, -1.09665089, -0.65474624, ..., -1.01952389,
         1.76660101, -0.57375783],
       [ 0.35837243,  0.91186722, -0.55861083, ..., -1.01952389,
         1.76660101, -0.57375783],
       ...,
       [ 1.06083725,  0.91186722, -0.84701706, ..., -1.01952389,
        -0.56605877,  1.7428956 ],
       [-1.04655719, -1.09665089, -1.32769411, ...,  0.98084999,
        -0.56605877, -0.57375783],
       [-2.18678992, -1.09665089,  0.01820164, ...,  0.98084999,
        -0.56605877, -0.57375783]])

In [18]:
import tensorflow as tf

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense

from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [19]:
model= Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(32,activation='relu' ) ,
    Dense(1,activation='sigmoid' )   
])

C:\Users\isdcb\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
optizer=tf.keras.optimizers.Adam(learning_rate=0.001)

In [21]:
model.compile(optimizer=optizer,
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [22]:
import datetime

In [23]:
#SETUP TENSORBOARD
from tensorflow.keras.callbacks import EarlyStopping,Callback,TensorBoard

log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflow_callback=TensorBoard(log_dir,histogram_freq=1)

# set up early_stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5)

In [24]:
history =model.fit(x_train,y_train,epochs=10,batch_size=40,
          validation_data=(x_test,y_test),validation_batch_size=50,
          callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7863 - loss: 0.4965 - val_accuracy: 0.8237 - val_loss: 0.4190
Epoch 2/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8284 - loss: 0.4081 - val_accuracy: 0.8423 - val_loss: 0.3929
Epoch 3/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8467 - loss: 0.3750 - val_accuracy: 0.8513 - val_loss: 0.3668
Epoch 4/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8572 - loss: 0.3489 - val_accuracy: 0.8543 - val_loss: 0.3566
Epoch 5/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8579 - loss: 0.3522 - val_accuracy: 0.8557 - val_loss: 0.3532
Epoch 6/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8649 - loss: 0.3341 - val_accuracy: 0.8577 - val_loss: 0.3534
Epoch 7/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8647 - loss: 0.3335 - val_accuracy: 0.8590 - val_loss: 0.3532
Epoch 8/10
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8636 - loss: 0.3303 - val_accuracy: 0.

In [25]:
model.save('model.h5')

In [26]:
# load tensorboard extension
%load_ext tensorboard

In [27]:
%tensorboard --logdir logs/fit20250227-233510

Reusing TensorBoard on port 6007 (pid 6280), started 21:17:17 ago. (Use '!kill 6280' to kill it.)